# `gu_toolkit` overview (updated)

This notebook is a hands-on tour of the current API surface.

It is organized so you can **test behavior directly**:
- each section says **what to do**,
- and what you should **expect to happen**.


In [ ]:
# Setup
import sys
from pathlib import Path

try:
    _start = Path(__file__).resolve().parent
except NameError:
    _start = Path.cwd().resolve()

_pkg_root = _start
while _pkg_root != _pkg_root.parent and not (_pkg_root / "__init__.py").exists():
    _pkg_root = _pkg_root.parent
sys.path.insert(0, str(_pkg_root.parent))


from gu_toolkit import *

## 1) Discoverability and introspection

**Do this:** run the next cells.

**Expect:**
- a quick map of major notebook-facing names,
- style options for `plot`,
- signatures/docstrings you can inspect interactively.


In [ ]:
# Top-level exported names (sample)
public_names = [name for name in dir() if not name.startswith("_")]
[name for name in public_names if name in {
    "Figure", "plot", "parameter", "params", "plots", "render",
    "numpify", "parse_latex", "NIntegrate", "NReal_Fourier_Series", "play"
}]


In [ ]:
plot_style_options()


In [ ]:
import inspect

print(inspect.signature(Figure.plot))
print(inspect.signature(numpify))
print(inspect.signature(NIntegrate))


## 2) Core plotting workflow (context-managed)

**Do this:** execute the next cell, then pan/zoom.

**Expect:**
- multiple traces appear on one figure,
- updating an existing `id` replaces that trace rather than creating a duplicate,
- re-renders happen as you change view ranges.


In [ ]:
fig1 = Figure(x_range=(-6, 6), y_range=(-2.5, 2.5), sampling_points=500)
display(fig1)

with fig1:
    set_title("Trigonometric overlays")
    plot(x, sin(x), id="sin")
    plot(x, cos(x), id="cos", dash="dash", color="#d62728")
    plot(x, sin(2*x), id="sin2", opacity=0.4, thickness=4)


In [ ]:
# Reusing id="sin" updates that trace.
with fig1:
    plot(x, sin(x**2/3), id="sin", color="#1f77b4", thickness=2)


## 3) Parameter sliders (automatic and explicit)

**Do this:** run the cells and move sliders in the right panel.

**Expect:**
- parameters are auto-discovered from free symbols,
- manual slider registration works too,
- figure updates immediately when sliders move.


In [ ]:
fig2 = Figure(x_range=(-8, 8), y_range=(-4, 4))
display(fig2)

with fig2:
    set_title("Auto-parameterized expression")
    plot(x, a*sin(b*x + c), id="wave")


In [ ]:
# Explicit slider defaults and ranges
with fig2:
    parameter(a, min=-3, max=3, value=1.0, step=0.05, readout_format=".2f")
    parameter(b, min=0.2, max=4.0, value=1.0, step=0.05)
    parameter(c, min=-3.14, max=3.14, value=0.0, step=0.01)


In [ ]:
# Current numeric parameter values
with fig2:
    params_snapshot = params.snapshot()
params_snapshot

## 4) Plot/figure introspection

**Do this:** inspect figure and plot objects.

**Expect:**
- discoverable plot ids,
- access to symbolic expression and live numeric expression,
- access to last sampled x/y arrays for debugging/analysis.


In [ ]:
list(fig2.plots.keys())


In [ ]:
wave = fig2.plots["wave"]
wave.symbolic_expression, wave.parameters


In [ ]:
# Numeric callable (parameter-aware through the figure context)
wave_numeric = wave.numeric_expression
wave_numeric(np.array([0.0, 1.0, 2.0]))


In [ ]:
# Last sampled arrays used for rendering (None before first render)
wave.x_data[:5], wave.y_data[:5]


## 5) Freezing, unfreezing, and parameter snapshots

**Do this:** run the cell and compare the outputs.

**Expect:**
- a snapshot captures current slider values,
- `freeze(...)` freezes numeric evaluation at chosen values,
- `unfreeze()` restores explicit function arguments for unfrozen parameters (pass current values each call).


In [ ]:
f_live = wave.numeric_expression

with wave.figure:
    snap = params.snapshot(full=True)
    
f_frozen = f_live.freeze(snap.value_map())

print("live @ x=1:", f_live(1.0))
print("frozen @ x=1:", f_frozen(1.0))

with wave.figure:
    params[a] = 2.0
    
print("after changing a...")
print("live @ x=1:", f_live(1.0))
print("frozen @ x=1 (unchanged):", f_frozen(1.0))

f_frozen = f_frozen.unfreeze()
with wave.figure:
    a_val = params[a].value
    b_val = params[b].value
    c_val = params[c].value
print("unfrozen now tracks call-time arguments again:", f_frozen(1.0, a_val, b_val, c_val))

with wave.figure:
    params[a] = 3.0
    a_val = params[a].value
    b_val = params[b].value
    c_val = params[c].value
print("after another slider update, pass refreshed values:", f_frozen(1.0, a_val, b_val, c_val))


## 6) NumericFunction + `numpify`: complete workflow (standalone and with figures)

This section shows three usage layers:

1. compile symbolic math with `numpify(...)` into a `NumericFunction`,
2. control parameters with `freeze(...)` / `unfreeze(...)`,
3. reuse the same semantics when a plot is managed by `Figure`.

**Key points**
- `numpify(...)` returns a `NumericFunction` object.
- The object is callable and NumPy-vectorized.
- `vars` can be positional, keyed, or mixed (including indexed mappings).
- Figure-backed `plot(...).numeric_expression` is also a `NumericFunction`.


In [ ]:
# 6a) Basic standalone compilation
expr = a*exp(-x**2) * cos(b*x)
f = numpify(expr, vars=(x, a, b))
print(type(f).__name__)
f(np.linspace(-2, 2, 5), 1.0, 2.0)


In [ ]:
# 6b) Freezing and unfreezing standalone callables
f_frozen = f.freeze({a: 1.5, b: 3.0})
print('frozen signature:', inspect.signature(f_frozen))
print('frozen eval:', f_frozen(np.linspace(-2, 2, 5)))

f_unfrozen = f_frozen.unfreeze()
print('unfrozen signature:', inspect.signature(f_unfrozen))
print('unfrozen eval:', f_unfrozen(np.linspace(-2, 2, 5), 1.5, 3.0))


In [ ]:
# 6c) Advanced vars contract: mixed positional + keyed arguments
f_mixed = numpify(x + a*b + c, vars=(x, {'alpha': a, 'beta': b, 'bias': c}))
print('vars positional view:', tuple(f_mixed.vars))
print('vars round-trip spec:', f_mixed.vars())
f_mixed(2.0, alpha=3.0, beta=4.0, bias=1.0)


In [ ]:
# 6d) Indexed mapping form for vars (0..n-1 are positional slots)
y, scale = symbols('y scale')
f_indexed = numpify(x + y*scale, vars={0: x, 1: y, 'scale': scale})
print('vars positional view:', tuple(f_indexed.vars))
print('vars round-trip spec:', f_indexed.vars())
f_indexed(2.0, 3.0, scale=4.0)


### 6e) NumericFunction in conjunction with `Figure`

When you create symbolic plots, `plot_obj.numeric_expression` exposes the compiled `NumericFunction` tied to the figure parameter context.
This is useful when you want both interactive plotting and direct numeric access in the same workflow.


In [ ]:
fig_nf = Figure(x_range=(-4, 4), y_range=(-3, 3), sampling_points=300)
with fig_nf:
    p = plot(a*sin(x) + b*cos(2*x), label='combo')

g = p.numeric_expression
print(type(g).__name__)
print('signature before freeze:', inspect.signature(g))
g(np.array([0.0, 1.0]))


In [ ]:
# Figure parameter context drives live evaluation for non-x symbols
with fig_nf:
    params[a] = 1.25
    params[b] = -0.5

g_live = g.freeze({a: DYNAMIC_PARAMETER, b: DYNAMIC_PARAMETER})
print('signature with dynamic params:', inspect.signature(g_live))
g_live(np.array([0.0, 1.0]))


In [ ]:
# Freeze current figure state into a standalone numeric function
with fig_nf:
    snap = params.snapshot(full=True)
g_snapshot = g.freeze(snap.value_map())

x_sample = np.linspace(-2, 2, 6)
y_from_snapshot = g_snapshot(x_sample)
y_from_direct = numpify(a*sin(x) + b*cos(2*x), vars=(x, a, b)).freeze(snap.value_map())(x_sample)
np.allclose(y_from_snapshot, y_from_direct)


You can also use `numpify` for constants (remember: it still returns a callable).


In [ ]:
numpify(pi**(1/2))()


## 7) Parse LaTeX into SymPy, then plot

**Do this:** parse, inspect, and plot.

**Expect:**
- LaTeX turns into a SymPy expression,
- expression can be plotted like any other symbolic input.


In [ ]:
expr = parse_latex(r"\sin(x) + \frac{1}{2}\cos(3x)")
expr


In [ ]:
fig3 = Figure(x_range=(-6, 6), y_range=(-2, 2))
display(fig3)
with fig3:
    set_title("Expression parsed from LaTeX")
    plot(x, expr, id="latex_expr", color="#2ca02c")

#BUG HERE:  AttributeError: 'Tree' object has no attribute 'free_symbols'
#PROBLEM: the parser seturns a Tree and it is ambiguious. This is bullshit The expression $\sin(x) + \frac{1}{2}\cos(3x)$ is absolutely unambiguous
#Tree('_ambig', [sin(x + cos(3*x)/2), sin(x + 1/2)*cos(3*x), sin(x) + cos(3*x)/2])

## 8) Numeric helpers from the prelude

### 8.1 `NIntegrate`

**Do this:** integrate a symbolic expression.

**Expect:**
- a scalar float close to known analytic values.


In [ ]:
print(
    numpify(pi**(1/2))()
)
print(
    NIntegrate(exp(-x**2), (x, -oo, oo))  # should be close to sqrt(pi)
     )

### 8.2 `NReal_Fourier_Series`

**Do this:** compute Fourier coefficients on a finite interval.

**Expect:**
- two arrays `(cos_coeffs, sin_coeffs)`; index 0 is the DC component.


In [ ]:
cos_coeffs, sin_coeffs = NReal_Fourier_Series(sin(x) + 0.3*cos(2*x), (x, -pi, pi), samples=2048)
cos_coeffs[:6], sin_coeffs[:6]


### 8.3 `play`

**Do this:** evaluate the cell.

**Expect:**
- an audio widget appears and autoplays,
- use `loop=False` for one-shot playback.


In [ ]:
play(sin(2*pi*220*x) * exp(-2*x), (x, 0, 1.5), loop=False)
#BUG, autoplays without display! Should not do by default unless autoplay=True)


## 9) Common examples you can adapt quickly

These are compact patterns you can copy for day-to-day use.


In [ ]:
# Example A: Compare two models with one shared parameter
figA = Figure(x_range=(-4, 4), y_range=(-2, 2))
display(figA)
with figA:
    plot(x, tanh(k*x), id="tanh", label="tanh(kx)")
    plot(x, x/sqrt(1 + (k*x)**2), id="softsign", label="softsign-like", dash="dot")
    parameter(k, min=0.1, max=5, value=1.0, step=0.1)


In [ ]:
with figA:
    print([k for k in plots.keys()])

In [ ]:
# Example B: Build expressions with indexed symbol families
expr_family = a[1]*sin(x) + a[2]*cos(2*x) + a[3]*sin(3*x)
expr_family


In [ ]:
figB = Figure(x_range=(-6, 6), y_range=(-4, 4))
display(figB)
with figB:
    plot(x, expr_family, id="family")
    parameter(a[1], min=-2, max=2, value=1)
    parameter(a[2], min=-2, max=2, value=0.5)
    parameter(a[3], min=-2, max=2, value=0.2)


## 10) Simplified Info Cards (`fig.info`)

The new Info Card API gives you a concise way to render rich (HTML + LaTeX) sidebar content that can mix static and dynamic segments.

### API recap
- `fig.info(spec, id=None)`
- `spec` can be:
  - a single string
  - a callable `(fig, ctx) -> str`
  - a sequence combining strings and callables
- `id` semantics:
  - omit `id` to auto-create cards (`info0`, `info1`, ...)
  - reuse an existing `id` to replace that card in place

### Update behavior
Dynamic callables are evaluated on all render reasons:
- `manual` (`fig.render()`),
- `param_change` (slider interactions),
- `relayout` (pan/zoom redraw path).

Callables receive a context object with fields:
- `ctx.reason`
- `ctx.trigger`
- `ctx.t` (timestamp)
- `ctx.seq` (monotone update id)

Errors raised inside dynamic callables are rendered in-place as a bounded, escaped traceback block so the notebook layout remains stable.

In [ ]:
# Static card: rendered once (supports HTML + LaTeX)
fig_info = Figure(x_range=(-8, 8), y_range=(-3, 3))
x, a = sp.symbols('x a')
fig_info.parameter(a, min=-3, max=3, value=1.0, step=0.1)
fig_info.plot(x, a*sp.sin(x), id='wave', label='a*sin(x)')

fig_info.info(
    "<b>Model</b>: $y = a\\sin(x)$"
)
fig_info

In [ ]:
# Mixed static + dynamic segments in one ordered card
def range_summary(fig, ctx):
    xr = tuple(round(v, 3) for v in fig.x_range)
    yr = tuple(round(v, 3) for v in fig.y_range)
    return (
        f"<div><b>reason</b>: <code>{ctx.reason}</code></div>"
        f"<div><b>x_range</b>: <code>{xr}</code></div>"
        f"<div><b>y_range</b>: <code>{yr}</code></div>"
    )

def parameter_summary(fig, ctx):
    aval = fig.parameters[a].value
    return f"<div><b>a</b>: <code>{aval:.3f}</code></div>"

fig_info.info([
    "<hr style='margin:0.3rem 0;'/>",
    "<b>Live Diagnostics</b>",
    range_summary,
    parameter_summary,
], id='diagnostics')

# Trigger a manual update explicitly (dynamic segments also update automatically on slider drag/pan/zoom)
fig_info.render(reason='manual')
fig_info

In [ ]:
# Replace an existing card in-place by reusing id='diagnostics'
fig_info.info([
    "<b>Diagnostics (replaced card content)</b>",
    lambda fig, ctx: f"<div>seq=<code>{ctx.seq}</code>, reason=<code>{ctx.reason}</code></div>",
], id='diagnostics')
fig_info.render(reason='manual')
fig_info

In [ ]:
# Module-level helper also works inside a figure context
with fig_info:
    info([
        "<b>Module-level info helper</b>",
        lambda fig, ctx: f"<div>trigger type: <code>{type(ctx.trigger).__name__}</code></div>",
    ], id='module-helper')

fig_info.render(reason='manual')
fig_info

In [ ]:
# Error handling demo: exceptions render as bounded escaped <pre> blocks
def unstable_segment(fig, ctx):
    raise RuntimeError("<b>example failure</b>: dynamic segment crashed")

fig_info.info(["<b>Error card demo</b>", unstable_segment], id='error-demo')
fig_info.render(reason='manual')
fig_info

## 11) What to explore next

- Use `help(Figure)` and `help(Figure.plot)` for full docstrings.
- Inspect `fig.plots[<id>]` objects to understand data flow from symbolic to numeric.
- Explore the `tests/` and `documentation/develop_guide/` folders for deeper patterns.
